In [ ]:
#!pip install opencv-python pillow
#!pip install torch torchvision


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Preprocessing

In [ ]:
# Set paths to your dataset
faded_image_dir = '/content/drive/MyDrive/faded'
clean_image_dir = '/content/drive/MyDrive/trainB'


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import os

class ImageDataset(Dataset):
    def __init__(self, noisy_dir, clean_dir, transform=None):
        self.noisy_dir = noisy_dir
        self.clean_dir = clean_dir
        self.transform = transform
        self.noisy_images = sorted(os.listdir(noisy_dir))
        self.clean_images = sorted(os.listdir(clean_dir))

    def __len__(self):
        return len(self.noisy_images)

    def __getitem__(self, idx):
        noisy_path = os.path.join(self.noisy_dir, self.noisy_images[idx])
        clean_path = os.path.join(self.clean_dir, self.clean_images[idx])
        noisy_img = Image.open(noisy_path).convert('RGB')
        clean_img = Image.open(clean_path).convert('RGB')

        if self.transform:
            noisy_img = self.transform(noisy_img)
            clean_img = self.transform(clean_img)

        return noisy_img, clean_img

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Load dataset
dataset = ImageDataset(faded_image_dir, clean_image_dir, transform=transform)

# Split dataset into training and validation
train_size = int(0.8 * len(dataset))  # 80% training
val_size = len(dataset) - train_size  # 20% validation
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoader for training and validation
train_loader1 = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader1 = DataLoader(val_dataset, batch_size=16, shuffle=False)


#Model

In [ ]:
import torch.nn as nn

# Define the DnCNN model
class DnCNN(nn.Module):
    def __init__(self, channels=3, num_of_layers=17):
        super(DnCNN, self).__init__()
        kernel_size = 3
        padding = 1
        features = 64
        layers = []

        layers.append(nn.Conv2d(in_channels=channels, out_channels=features, kernel_size=kernel_size, padding=padding, bias=False))
        layers.append(nn.ReLU(inplace=True))

        for _ in range(num_of_layers-2):
            layers.append(nn.Conv2d(in_channels=features, out_channels=features, kernel_size=kernel_size, padding=padding, bias=False))
            layers.append(nn.BatchNorm2d(features))
            layers.append(nn.ReLU(inplace=True))

        layers.append(nn.Conv2d(in_channels=features, out_channels=channels, kernel_size=kernel_size, padding=padding, bias=False))
        self.dncnn = nn.Sequential(*layers)

    def forward(self, x):
        out = self.dncnn(x)
        return x - out  # Residual learning


#Training

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model1 = DnCNN().to(device)


# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model1.parameters(), lr=1e-4)

# Training and validation loop
num_epochs = 10
for epoch in range(num_epochs):
    model1.train()
    train_loss = 0
    for data in train_loader1:
        noisy_img, clean_img = data
        noisy_img = noisy_img.to(device)
        clean_img = clean_img.to(device)

        # Forward pass
        optimizer.zero_grad()
        output = model1(noisy_img)
        loss = criterion(output, clean_img)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation step
    model1.eval()
    val_loss = 0
    with torch.no_grad():
        for data in val_loader1:
            noisy_img, clean_img = data
            noisy_img = noisy_img.to(device)
            clean_img = clean_img.to(device)

            output = model1(noisy_img)
            loss = criterion(output, clean_img)
            val_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss/len(train_loader1)}, Val Loss: {val_loss/len(val_loader1)}")


KeyboardInterrupt: 

In [ ]:
import torch
import os

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the model and move to device
model1 = DnCNN().to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model1.parameters(), lr=1e-4)

# Load previous checkpoint (if exists)
checkpoint_path = '/content/drive/MyDrive/models/dncnn_finetuned_faded_epoch_15.pth'
optimizer_checkpoint_path = '/content/drive/MyDrive/models/optimizer_finetuned_epoch_15.pth'

start_epoch = 0  # Default start epoch
if os.path.exists(checkpoint_path):
    model1.load_state_dict(torch.load(checkpoint_path))
    print("Model loaded from", checkpoint_path)

    if os.path.exists(optimizer_checkpoint_path):
        optimizer.load_state_dict(torch.load(optimizer_checkpoint_path))
        print("Optimizer state loaded from", optimizer_checkpoint_path)

    # Update start_epoch if you saved the epoch number as part of the checkpoint
start_epoch = 16
# Training and validation loop
num_epochs = 20  # Adjust the number of epochs as needed
for epoch in range(start_epoch, num_epochs):
    model1.train()
    train_loss = 0
    for data in train_loader1:
        noisy_img, clean_img = data
        noisy_img = noisy_img.to(device)
        clean_img = clean_img.to(device)

        # Forward pass
        optimizer.zero_grad()
        output = model1(noisy_img)
        loss = criterion(output, clean_img)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation step
    model1.eval()
    val_loss = 0
    with torch.no_grad():
        for data in val_loader1:
            noisy_img, clean_img = data
            noisy_img = noisy_img.to(device)
            clean_img = clean_img.to(device)

            output = model1(noisy_img)
            loss = criterion(output, clean_img)
            val_loss += loss.item()

    # Print losses for this epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss/len(train_loader1)}, Val Loss: {val_loss/len(val_loader1)}")

    # Save the model and optimizer state after each epoch
    torch.save(model1.state_dict(), f'/content/drive/MyDrive/models/dncnn_finetuned_faded_epoch_{epoch+1}.pth')
    torch.save(optimizer.state_dict(), f'/content/drive/MyDrive/models/optimizer_finetuned_epoch_{epoch+1}.pth')
    print(f"Model and optimizer saved after epoch {epoch+1}")


<ipython-input-13-3269b95373c1>:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model1.load_state_dict(torch.load(checkpoint_path))


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
# prompt: how can i visualize the output

import matplotlib.pyplot as plt
import torchvision.transforms as T

model1.eval()
with torch.no_grad():
    for data in val_loader1:
        noisy_img, clean_img = data
        noisy_img = noisy_img.to('cuda')

        output = model1(noisy_img)

        # Move images to CPU and convert to NumPy arrays
        noisy_img_np = noisy_img[1].cpu().permute(1, 2, 0).numpy()
        output_np = output[1].cpu().permute(1, 2, 0).numpy()
        clean_img_np = clean_img[1].cpu().permute(1, 2, 0).numpy()

        # Display images
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))
        axs[0].imshow(noisy_img_np)
        axs[0].set_title('Faded Image')
        axs[1].imshow(output_np)
        axs[1].set_title('Model Output')
        axs[2].imshow(clean_img_np)
        axs[2].set_title('Ground Truth Image')
        plt.show()
        break  # Display only the first batch


In [ ]:
import os

# Create the directory if it doesn't exist
model_dir = '/content/drive/MyDrive/models'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Now save the model
torch.save(model1.state_dict(), '/content/drive/MyDrive/models/dncnn_finetuned_faded.pth')

#Testing

In [ ]:
import torch
import os
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image

# Load the model
model = DnCNN()
model.load_state_dict(torch.load('/content/drive/MyDrive/models/dncnn_finetuned_faded.pth', map_location=torch.device('cpu')))
model.eval()  # Set the model to evaluation mode

# Load dataset
faded_image_dir = '/content/drive/MyDrive/test_faded'
clean_image_dir = '/content/drive/MyDrive/test_restored'

# Define image transformations for resizing
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize to 256x256 for model input
    transforms.ToTensor()
])

# Create dataset and DataLoader for testing
test_dataset = ImageDataset(faded_image_dir, clean_image_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Create a folder to save the output images
output_folder = '/content/drive/MyDrive/test_results_faded'
os.makedirs(output_folder, exist_ok=True)

# Testing loop
for idx, (noisy_images, clean_images) in enumerate(test_loader):
    with torch.no_grad():
        output_images = model(noisy_images)  # Pass the batch through your model

    # Save the output images
    for i in range(output_images.size(0)):
        output_img = output_images[i].permute(1, 2, 0).numpy()  # Convert to HWC format
        output_img = (output_img * 255).clip(0, 255).astype('uint8')  # Scale back to [0, 255]

        # Get the original noisy image filename for correct naming
        noisy_image_name = test_dataset.noisy_images[idx * test_loader.batch_size + i]

        # Load the original image to get its size
        original_image_path = os.path.join(faded_image_dir, noisy_image_name)
        original_image = Image.open(original_image_path)
        original_size = original_image.size  # Get original size (width, height)

        # Resize the output image back to the original size
        output_img_resized = Image.fromarray(output_img).resize(original_size, Image.BICUBIC)

        # Construct the new filename as result<original_name>.png
        new_image_name = f'result_{noisy_image_name}'

        # Save the output image
        output_image_path = os.path.join(output_folder, new_image_name)
        output_img_resized.save(output_image_path)

print(f'Testing completed. Output images saved in {output_folder}')


<ipython-input-18-60c3b0297c18>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/models/dncnn_finetuned_faded.pth', 

Testing completed. Output images saved in /content/drive/MyDrive/test_results_faded


# Evaluation

### PNSR and SSIM

In [ ]:
from skimage.metrics import peak_signal_noise_ratio as psnr
import numpy as np
import os
from PIL import Image

# Load dataset for PSNR calculation
clean_image_dir = '/content/drive/MyDrive/test_restored'
output_folder = '/content/drive/MyDrive/test_results_faded'

clean_images_list = sorted(os.listdir(clean_image_dir))
output_images_list = sorted(os.listdir(output_folder))

# Variables to store PSNR results
psnr_values = []

# PSNR calculation loop
for i in range(len(output_images_list)):
    # Load clean and output images
    clean_img = Image.open(os.path.join(clean_image_dir, clean_images_list[i])).convert('RGB')
    output_img = Image.open(os.path.join(output_folder, output_images_list[i])).convert('RGB')

    # Convert images to numpy arrays
    clean_img_np = np.array(clean_img)
    output_img_np = np.array(output_img)

    # Calculate PSNR
    psnr_value = psnr(clean_img_np, output_img_np)
    psnr_values.append(psnr_value)
# Calculate average PSNR
avg_psnr = np.mean(psnr_values)

print(f'Average PSNR: {avg_psnr:.2f}')


Average PSNR: 18.70


# Inference

In [ ]:
import torch
import os
from PIL import Image
import torchvision.transforms as transforms

# Load the saved model for inference
model_dir = '/content/drive/MyDrive/models/dncnn_finetuned_faded_epoch_15.pth'

# Instantiate the model
model = DnCNN()

# Load the model weights
model.load_state_dict(torch.load(model_dir, map_location=torch.device('cpu')))
model.eval()  # Set the model to evaluation mode

# Define preprocessing transformation for the input image
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize for the model
    transforms.ToTensor(),           # Convert image to tensor
])

# Define a function for inference
def infer(image_path, output_path):
    # Load and preprocess the input image
    input_image = Image.open(image_path).convert('RGB')
    input_tensor = transform(input_image).unsqueeze(0)  # Add batch dimension

    # Run the model on the input image
    with torch.no_grad():
        output_tensor = model(input_tensor)

        # Convert output tensor to numpy array and scale back to [0, 255]
        output_img = output_tensor.squeeze().permute(1, 2, 0).numpy()  # Convert to HWC format
        output_img = (output_img * 255).clip(0, 255).astype('uint8')  # Scale back to [0, 255]

        # Get the original image size
        original_size = input_image.size  # Get original size (width, height)

        # Resize the output image back to the original size
        output_img_resized = Image.fromarray(output_img).resize(original_size, Image.BICUBIC)

    # Save the output image
    output_img_resized.save(output_path)
    print(f"Output image saved to: {output_path}")

    return output_img_resized  # Return the processed image

# Example of using the infer function
input_image_path = '/content/F3103.png'  # Path to the input image
output_image_path = '/content/result_15.png'  # Path to save the output image
output_image = infer(input_image_path, output_image_path)  # Run inference and save the output image

# To display the output image (optional)
output_image.show()


<ipython-input-8-65c152e53cbc>:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_dir, map_location=torch.device('cpu')))


Output image saved to: /content/result_15.png
